In [1]:
import os

from langchain.agents import Tool
from langchain.memory import ChatMessageHistory, ConversationBufferWindowMemory
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.agents import AgentExecutor
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datetime import datetime
hoje = datetime.today()

data_formatada = hoje.strftime("%Y/%m/%d")
# Pega o dia da semana em inglês
dia_da_semana_ingles = hoje.strftime("%A")

# Dicionário para mapear os dias da semana de inglês para português
dias_da_semana_pt = {
    "Monday": "segunda-feira",
    "Tuesday": "terça-feira",
    "Wednesday": "quarta-feira",
    "Thursday": "quinta-feira",
    "Friday": "sexta-feira",
    "Saturday": "sábado",
    "Sunday": "domingo"
}

# Traduz o dia da semana para português
dia_da_semana_pt = dias_da_semana_pt.get(dia_da_semana_ingles, dia_da_semana_ingles)

# Formata a data para utilizar no prompt
data_atual = f"{dia_da_semana_pt}, {data_formatada}"

In [3]:
#(https://makersuite.google.com/)
from dotenv import load_dotenv

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')

In [4]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    convert_system_message_to_human=True,
    handle_parsing_errors=True,
    temperature=0.6,
    max_tokens= 1000,
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    },
)

In [5]:
def transfer_to_calendar_agent(input_str):
    return calendar_agent_executor.invoke({"input": input_str})
def transfer_to_travel_agent(input_str):
    return travel_agent_executor.invoke({"input": input_str})

In [6]:
from calendar_tools import extract_event_parameters, insert_calendar_event
print(insert_calendar_event("""
**Dia 11 de Janeiro de 2025 (Sexta-feira):**

* **Noite (18:00 - 22:00):** Jantar em um restaurante com culinária potiguar.  Pesquise opções na região de Ponta Negra ou no centro de Natal. Após o jantar, aproveite para um passeio noturno pela orla de Ponta Negra.
"""))

calendar v3 service created successfully


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


{'kind': 'calendar#event', 'etag': '"3473087252100000"', 'id': 'tvvpd3nb2eroj7186oo133j6h4', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=dHZ2cGQzbmIyZXJvajcxODZvbzEzM2o2aDQgdGFsZXMudi5tLmFsdmVzQG0', 'created': '2025-01-10T21:13:46.000Z', 'updated': '2025-01-10T21:13:46.050Z', 'summary': 'Jantar e passeio em Natal', 'description': 'Jantar em um restaurante com culinária potiguar em Ponta Negra ou no centro de Natal. Após o jantar, passeio noturno pela orla de Ponta Negra.', 'location': 'Restaurante em Ponta Negra ou Centro de Natal + Orla de Ponta Negra', 'creator': {'email': 'tales.v.m.alves@gmail.com', 'self': True}, 'organizer': {'email': 'tales.v.m.alves@gmail.com', 'self': True}, 'start': {'dateTime': '2025-01-11T18:00:00-03:00', 'timeZone': 'America/Fortaleza'}, 'end': {'dateTime': '2025-01-11T22:00:00-03:00', 'timeZone': 'America/Fortaleza'}, 'iCalUID': 'tvvpd3nb2eroj7186oo133j6h4@google.com', 'sequence': 0, 'reminders': {'useDefault': True}, 'ev

In [7]:
from planing_tools import weatherapi_forecast_periods, query_rag
ddg_search = DuckDuckGoSearchAPIWrapper()

travel_planing_tools = [
    Tool(
        name="DuckDuckGo Search",
        func=ddg_search.run,
        description="""Essa ferramenta DEVE ser utilizada para buscar eventos relevantes no período fornecido pelo usuário. 
        Ela é útil para obter informações sobre eventos ou atividades especiais que estão acontecendo na cidade de destino nas datas que o usuário informou. 
        O modelo deve usá-la para complementar as sugestões de atividades."""
    ),
    Tool(
        name="Weather Forecast",
        func=weatherapi_forecast_periods,
        description="""Essa ferramenta DEVE ser usada *antes* de gerar o roteiro turístico, após o modelo coletar todas as informações necessárias do usuário, incluindo as datas exatas. 
    O modelo deve consultar o clima separadamente para cada dia do período informado, garantindo que as atividades do roteiro sejam planejadas de acordo com as condições climáticas. 
    A previsão do tempo é um passo obrigatório quando é um fator relevante para o planejamento."""
    ),
    Tool(
        name="Query RAG",
        func=query_rag,
        description="""Esta ferramenta deve ser usada quando o modelo souber a cidade de destino e os interesses do usuário, com o objetivo de fornecer informações sobre pontos turísticos e atrações que se alinham com esses interesses. 
        O modelo deve utilizar essa ferramenta para sugerir atividades e lugares específicos a visitar, baseados na cidade e nos interesses fornecidos."""
    ),
    Tool(
        name="Calendar Agent",
        func=transfer_to_calendar_agent,
        description="Esse agente lida com tudo relacionado ao calendário."
    ),
]

In [8]:
from calendar_tools import list_calendar_list, list_calendar_events, insert_calendar_event, create_calendar

google_calendar_tools = [
    Tool(
        name="Create Calendar List",
        func=create_calendar,
        description="""
        Use a função create_calendar_list para criar um novo calendário.
        Forneça o resumo (título) do calendário que você deseja criar.
        - Exemplo de uso: 'Meu Calendário de Viagens'
        - Essa função vai criar um novo calendário com o resumo fornecido.
        """
    ),
    Tool(
        name="List Calendar List",
        func=list_calendar_list,
        description="""
        Use a função list_calendar_list para obter a lista dos seus calendários disponíveis no Google Calendar.
        Se você quiser limitar o número de calendários retornados, forneça o número máximo desejado.
        - Exemplo de uso: Forneça o número máximo de calendários a serem listados, como 50. Se não quiser um limite, apenas diga para listar 200 o máximo por padrão.
        """
    ),
    Tool(
        name="List Calendar Events",
        func=list_calendar_events,
        description="""
        Use a função list_calendar_events para obter a lista de eventos de um calendário específico.
        Você precisa fornecer o ID do calendário e, opcionalmente, o número máximo de eventos a serem listados.
        - Exemplo de uso para o calendário principal: forneça 'primary' como o ID do calendário.
        - Exemplo de uso com limite: forneça o ID do calendário e o número máximo de eventos, como 'primary', 20.
        - Para listar eventos de um calendário específico, primeiro liste seus calendários para encontrar o ID correto.
        """
    ),
    Tool(
    name="Insert Calendar Event",
    func=insert_calendar_event,
    description="""
        Use a função Insert Calendar Event para adicionar **um único evento** a um calendário Google.
        Esta função deve ser chamada **uma vez para cada evento** que precisa ser agendado.

        A entrada para esta função deve ser uma descrição clara e completa de **um único evento**
        em linguagem natural, contendo todas as informações necessárias para agendá-lo individualmente.

        Se o usuário especificar um calendário diferente do principal, você DEVE chamar
        a função 'list_calendar_list' **antes de chamar esta função** para obter o 'calendar_id' correto.

        **Formato de entrada esperado:**

        **Descrição em Linguagem Natural (obrigatório):**
        Forneça uma descrição textual detalhada para **um único evento**. Inclua o que será feito,
        quando (data e hora específicas), onde (local exato) e quaisquer detalhes adicionais relevantes
        para esse evento em particular.

        **Exemplos de descrições de eventos individuais:**
        - "Agendar almoço com a Maria na sexta-feira ao meio-dia no restaurante X."
        - "Criar um compromisso para pagar as contas no dia 10 de janeiro às 9h da manhã."
        - "Bloquear minha agenda para foco no projeto Y na próxima segunda-feira, das 14h às 17h, na minha mesa."

        **Como usar esta função corretamente:**

        1. **Identifique cada evento individualmente:** Se a intenção é agendar múltiplos eventos,
           trate cada um como uma chamada separada para esta função.
        2. **Obtenha o ID do calendário (se necessário):** Se o usuário mencionar um nome de
           calendário específico, use a função 'list_calendar_list' **primeiro** para obter
           o 'calendar_id' correspondente.
        3. **Chame esta função com a descrição de um único evento:** Forneça a descrição completa
           em linguagem natural para o evento específico que você deseja agendar.

        **Informações essenciais para cada descrição de evento:**
        - **O que:** Uma descrição concisa do evento (ex: "Reunião de equipe", "Consulta médica").
        - **Quando:** A data e hora de início e fim **precisas** do evento.
        - **Onde:** O local **específico** do evento.
        - **Fuso Horário (opcional):** Se aplicável, mencione o fuso horário se for diferente
          de "America/Fortaleza".

        **Exemplo de uso (para o agente):**

        *Usuário: Agendar visita ao Aquário Natal sábado das 9h às 12h.*
        *Agente:* (Chama 'Insert Calendar Event' com a Action Input: "Agendar visita ao Aquário Natal sábado das 9h às 12h.")

        **Observação IMPORTANTE:** Esta função **NÃO** deve ser usada para criar múltiplos eventos
        com uma única chamada. Cada chamada deve corresponder a **um único evento**.

        Ao final da execução desta função, você DEVE fornecer ao usuário a confirmação do agendamento
        desse evento específico, incluindo o título, data, hora e link do evento criado (se disponível).
    """
    )
]

In [9]:
travel_planing_prompt = hub.pull("tales/agente_turismo")
google_calendar_prompt = hub.pull("tales/agente_calendario")
# main_agent_prompt = hub.pull("tales/agente_principal")

In [10]:
planing_prompt = travel_planing_prompt.partial(
    tools=render_text_description(travel_planing_tools),
    tool_names=", ".join([t.name for t in travel_planing_tools]),
)
llm_with_stop = llm.bind(stop=["\nObservation"])

In [11]:
calendar_prompt = google_calendar_prompt.partial(
    tools=render_text_description(google_calendar_tools),
    tool_names=", ".join([t.name for t in google_calendar_tools]),
)

In [12]:
history = ChatMessageHistory()
memory = ConversationBufferWindowMemory(k=20, chat_memory=history, memory_key="chat_history")

C:\Users\Tales\AppData\Local\Temp\ipykernel_5816\275711510.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=20, chat_memory=history, memory_key="chat_history")


In [13]:
travel_planing_agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
        "data_atual": lambda x: data_atual,
    }
    | planing_prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

travel_agent_executor = AgentExecutor(agent=travel_planing_agent, tools=travel_planing_tools, verbose=True, memory=memory)

In [14]:
google_calendar_agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
        "data_atual": lambda x: data_atual,
    }
    | calendar_prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

calendar_agent_executor = AgentExecutor(agent=google_calendar_agent, tools=google_calendar_tools, verbose=True, memory=memory)

In [15]:
response = travel_agent_executor.invoke({"input": 'Boa noite'})

print(response["output"])

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Boa noite!  Para começar a planejar sua viagem, preciso de algumas informações. Para onde você está planejando viajar?


> Finished chain.
Boa noite!  Para começar a planejar sua viagem, preciso de algumas informações. Para onde você está planejando viajar?


In [16]:
response = travel_agent_executor.invoke({"input": 'Eu estou planejando visitar Natal no Rio Grande do Norte'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? No
Final Answer: Ótimo! Natal é uma excelente escolha.  Quantos dias você pretende passar lá?


> Finished chain.
Ótimo! Natal é uma excelente escolha.  Quantos dias você pretende passar lá?


In [17]:
response = travel_agent_executor.invoke({"input": 'Eu planejo passar 2 dias da sábado dia 11 de janeiro ao domingo dia 12'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? No
Final Answer: Perfeito! Dois dias em Natal.  Agora, para te ajudar a criar o roteiro ideal, preciso saber quais atividades ou experiências você está buscando nessa viagem? Por exemplo, prefere explorar praias, restaurantes, eventos, museus, história, cultura, vida noturna, natureza, ou algo mais?


> Finished chain.
Perfeito! Dois dias em Natal.  Agora, para te ajudar a criar o roteiro ideal, preciso saber quais atividades ou experiências você está buscando nessa viagem? Por exemplo, prefere explorar praias, restaurantes, eventos, museus, história, cultura, vida noturna, natureza, ou algo mais?


In [18]:
response = travel_agent_executor.invoke({"input": 'Eu planejo visitar praias e restaurantes regionais'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? No
Final Answer: Entendido! Praias e restaurantes regionais em Natal.  Tem um orçamento em mente ou um limite de gastos que gostaria de seguir durante a viagem?


> Finished chain.
Entendido! Praias e restaurantes regionais em Natal.  Tem um orçamento em mente ou um limite de gastos que gostaria de seguir durante a viagem?


In [19]:
response = travel_agent_executor.invoke({"input": 'Eu ainda não sei'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? No
Final Answer: Tudo bem!  Vamos prosseguir sem um orçamento definido por enquanto.  Podemos ajustar o roteiro mais tarde, caso necessário. Você segue alguma dieta específica ou tem restrições alimentares, como alergias, que devo levar em consideração?


> Finished chain.
Tudo bem!  Vamos prosseguir sem um orçamento definido por enquanto.  Podemos ajustar o roteiro mais tarde, caso necessário. Você segue alguma dieta específica ou tem restrições alimentares, como alergias, que devo levar em consideração?


In [20]:
response = travel_agent_executor.invoke({"input": 'Não possuo nenhuma restrição alimentar.'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2025/01/11, 2025/01/12Previsão para 2025/01/11, 2025/01/12 em Natal:

Manhã:
Hora: 05:00 - Temp: 26.2°C, Condição: Aguaceiros fracos, Chance de chuva: 100%, Umidade: 82%
Hora: 06:00 - Temp: 27.4°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 75%
Hora: 07:00 - Temp: 28.4°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 81%, Umidade: 71%
Hora: 08:00 - Temp: 29.4°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 71%, Umidade: 66%
Hora: 09:00 - Temp: 30.2°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 63%
Hora: 10:00 - Temp: 30.4°C, Condição: Sol, Chance de chuva: 0%, Umidade: 61%
Hora: 11:00 - Temp: 30.9°C, Condição: Sol, Chance de chuva: 0%, Umidade: 59%

Tarde:
Hora: 13:00 - Temp: 30.7°C, Condição: Sol, Chance de chuva: 0%, Umidade: 58%
Hora: 14:00 - Temp: 30.1°C, Condição: Sol, Chance de chuva: 0%, Umidade

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Query RAG
Action Input: Atrações turísticas em Natal, Rio Grande do Norte, focadas em praias e restaurantes regionais.ATRAÇÕES
Em Natal, a vida além da 
praia garante aquário, museus, 
parque aquático, cajueiro 
gigante e estádio de futebol
PARQUE  
DAS DUNAS
Um dos grandes 
charmes do gigantesco 
parque é o Bosque dos 
Namorados, que tem 
área de piquenique, 
playground, restaurante e 
centro de visitantes. Vale 
a pena também percorrer 
as trilhas guiadas do 
parque, com vegetação 
exuberante. Embora 
não tenha praia, oferece 
mirantes com lindas vistas 
panorâmicas para o mar.
FOTOS: SHUTTERSTOCK
AQUÁRIO NATAL
Na Praia da Redinha, já no 
caminho para Genipabu, 
possui cerca de 60 
espécies de animais, como 
moreias, peixes de corais, 
cavalos-marinhos, jacarés e 
até mesmo mamíferos, como 
hipopótamos e jaguatiricas. 
Você pode passar a mão 
nas costas dos tubarões e 
chegar perto dos pinguins.

---

PRAIA A PRAIA
A orla urb

...

```tool_code
Thought: Do I need to use a tool? Yes
Action: DuckDuckGo Search
Action Input: eventos em Natal RN 11 e 12 de janeiro de 2025

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_community\utilities\duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


A programação cultural de janeiro 2025 em Natal será palco de grandes shows e eventos, garantindo diversão para moradores e turistas. Confira os destaques da programação cultural de janeiro 2025 em Natal e prepare-se para aproveitar o melhor da Cidade do Sol! No dia 11 de janeiro de 2025, a partir das 12h (meio-dia), o Rota 360 , realiza o evento Psy Sunset com várias atrações. Mais informações no perfil oficial @sigarota360. Encontre os melhores eventos no Brasil em 2025 na Sympla. Compre ingressos para shows, festas, teatros, congressos, eventos corporativos, esportivos, gastrônomicos, religiosos e mais! Descubra os melhores eventos em Natal, incluindo shows ao vivo, standups hilariantes, baladas animadas e muito mais. Fique por dentro de toda a diversão e entretenimento que Natal tem a oferecer. Acesse agora e planeje sua próxima experiência inesquecível na cidade! Compre ingressos para NUMANICE | NATAL em Natal dia 08/02. Confira os melhores eventos da Sympla!

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Olá! Com base nas informações que você me forneceu, preparei um roteiro para sua viagem a Natal nos dias 11 e 12 de janeiro de 2025, focando em praias e restaurantes regionais.  Lembre-se que a previsão do tempo indica possibilidade de chuva, principalmente na manhã do dia 11.  Sugiro verificar a previsão mais próxima da data da viagem para ter informações mais precisas.

**Roteiro de Viagem - Natal (RN)**

**Sábado, 11 de Janeiro de 2025**

* **Manhã (08:00 - 12:00):**  Considerando a previsão de chuva pela manhã, sugiro começar o dia visitando o Aquário Natal na Praia da Redinha.  Assim, você pode aproveitar as atividades internas e se proteger da chuva. Após a visita ao aquário, aproveite para explorar o Mercado Público da Redinha e experimentar a ginga com tapioca.
* **Tarde (13:00 - 17:00):**  Se o tempo estiver bom, aproveite a tarde na Praia de Ponta Negra.  Você pode relaxar na praia, tirar fotos do Morro do Careca

In [21]:
response = travel_agent_executor.invoke({"input": 'Eu quero que você salve os eventos sugeridos no meu calendário google'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Calendar Agent
Action Input: Save the following events to my Google Calendar:

**Saturday, January 11, 2025**

* **Morning (8:00 AM - 12:00 PM):** Visit the Natal Aquarium in Redinha Beach. Afterwards, explore the Redinha Public Market and try tapioca with ginga.
* **Afternoon (1:00 PM - 5:00 PM):** If the weather is good, relax at Ponta Negra Beach, take photos of Morro do Careca (from a distance), and enjoy the atmosphere. If it's raining, visit the Dunes Park, which offers trails and an ocean view.
* **Evening (6:00 PM - 10:00 PM):** Have dinner at a regional restaurant in Ponta Negra, with seafood options. Afterwards, check for events or relax at your hotel.  Check if the Psy Sunset event at Rota 360 is happening and if it interests you.


**Sunday, January 12, 2025**

* **Morning (9:00 AM - 12:00 PM):** Visit either Meio Beach or Artists' Beach, depending on your preference. Meio Beach is quieter, while Artists' Beach is b

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Insert Calendar Event
Action Input: "Visita ao Aquário Natal e Mercado Público da Redinha - Sábado, 11 de Janeiro de 2025, 8h às 12h. Local: Aquário Natal, Praia da Redinha, Natal, RN.  Após o Aquário, explorar o Mercado Público da Redinha e experimentar tapioca com ginga."

...

```tool_code
Thought: Do I need to use a tool? Yes
Action: Insert Calendar Event
Action Input: "Jantar em restaurante regional (fora de Ponta Negra) e retorno - Domingo, 12 de Janeiro de 2025, 18h às 22h. Local: Restaurante regional (fora de Ponta Negra), Natal, RN. Retorno ao hotel e preparação para viagem."

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


{'kind': 'calendar#event', 'etag': '"3473087410882000"', 'id': '3osev62p9qq44nqduld0k57fbo', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=M29zZXY2MnA5cXE0NG5xZHVsZDBrNTdmYm8gdGFsZXMudi5tLmFsdmVzQG0', 'created': '2025-01-10T21:15:05.000Z', 'updated': '2025-01-10T21:15:05.441Z', 'summary': 'Jantar em restaurante regional', 'description': 'Jantar em restaurante regional (fora de Ponta Negra) e retorno. Retorno ao hotel e preparação para viagem.', 'location': 'Restaurante regional (fora de Ponta Negra), Natal, RN', 'creator': {'email': 'tales.v.m.alves@gmail.com', 'self': True}, 'organizer': {'email': 'tales.v.m.alves@gmail.com', 'self': True}, 'start': {'dateTime': '2025-01-12T18:00:00-03:00', 'timeZone': 'America/Fortaleza'}, 'end': {'dateTime': '2025-01-12T22:00:00-03:00', 'timeZone': 'America/Fortaleza'}, 'iCalUID': '3osev62p9qq44nqduld0k57fbo@google.com', 'sequence': 0, 'reminders': {'useDefault': True}, 'eventType': 'default'}

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


ServiceUnavailable: 503 The model is overloaded. Please try again later.

In [ ]:
print(response["chat_history"])